In [3]:
import cv2
import numpy as np

def detect_shapes(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 50, 150)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        epsilon = 0.03 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)
        num_vertices = len(approx)

        area = cv2.contourArea(contour)
        perimeter = cv2.arcLength(contour, True)
        x, y, w, h = cv2.boundingRect(approx)
        aspect_ratio = float(w) / h
        extent = float(area) / (w * h)

        shape_name = 'Unknown'

        # Improved line detection
        if num_vertices == 2 or (num_vertices == 4 and (w / h > 10 or h / w > 10)):
            shape_name = 'Line'
        
        # Triangle detection
        elif num_vertices == 3:
            shape_name = 'Triangle'

        # Rectangle, square, and diamond detection
        elif num_vertices == 4:
            if 0.98 <= aspect_ratio <= 1.02:
                shape_name = 'Square'
            else:
                rect_area = w * h
                if abs(area - rect_area) / rect_area < 0.05:
                    shape_name = 'Rectangle'
                else:
                    _, _, angle = cv2.fitEllipse(contour)
                    if 40 < angle < 50 or 130 < angle < 140:
                        shape_name = 'Diamond'
                    else:
                        shape_name = 'Quadrilateral'

        # Pentagon detection
        elif num_vertices == 5:
            shape_name = 'Pentagon'

        # Hexagon detection
        elif num_vertices == 6:
            shape_name = 'Hexagon'

        # Circle and ellipse detection
        else:
            circularity = 4 * np.pi * area / (perimeter * perimeter) if perimeter > 0 else 0
            if circularity > 0.85:
                shape_name = 'Circle'
            else:
                try:
                    if len(contour) >= 5:
                        (_, axes, _) = cv2.fitEllipse(contour)
                        major, minor = max(axes), min(axes)
                        if major / minor < 1.1:
                            shape_name = 'Circle'
                        elif major / minor < 2:
                            shape_name = 'Ellipse'
                except:
                    pass

            # Star shape detection
            if shape_name == 'Unknown' and num_vertices > 8:
                hull = cv2.convexHull(contour)
                hull_area = cv2.contourArea(hull)
                solidity = float(area) / hull_area
                if solidity < 0.6:
                    distances = [cv2.pointPolygonTest(contour, (approx[i][0][0], approx[i][0][1]), True) for i in range(num_vertices)]
                    if max(distances) / min(distances) > 2:
                        shape_name = 'Star'

        # Regular polygon detection
        if shape_name == 'Unknown':
            angles = []
            for i in range(num_vertices):
                pt1 = approx[i][0]
                pt2 = approx[(i + 1) % num_vertices][0]
                pt3 = approx[(i + 2) % num_vertices][0]
                angle = np.abs(np.arctan2(pt3[1] - pt2[1], pt3[0] - pt2[0]) - 
                               np.arctan2(pt1[1] - pt2[1], pt1[0] - pt2[0]))
                angles.append(angle)
            if np.std(angles) < 0.15 and num_vertices > 6:
                shape_name = f'Regular {num_vertices}-gon'
            else:
                shape_name = 'Irregular Polygon'

        cv2.drawContours(image, [approx], 0, (0, 255, 0), 2)
        M = cv2.moments(contour)
        if M['m00'] != 0:
            cx = int(M['m10'] / M['m00'])
            cy = int(M['m01'] / M['m00'])
            cv2.putText(image, shape_name, (cx, cy), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    cv2.imshow('Detected Shapes', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

detect_shapes(r"D:\Adobe-gen\problems\problems\occlusion1_rec.png")